In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from keras import optimizers
#from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from keras.backend.tensorflow_backend import set_session

# mobilenetv2
from keras.applications import resnet50, vgg16, vgg19, xception, densenet, inception_v3, mobilenet, nasnet, inception_resnet_v2
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from keras.applications.resnet50 import preprocess_input
#from keras.applications.xception import preprocess_input
import os
import datetime
import json
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
import glob
import cv2
from random import shuffle

In [3]:
dog_path = 'C:\\tmp\\CatDogsData\\Dog\\*.jpg'
cat_path = 'C:\\tmp\\CatDogsData\\Cat\\*.jpg'

In [4]:
addrsd = glob.glob(dog_path)
addrsc = glob.glob(cat_path)

In [5]:
labelsd = [1 for addr in addrsd]  # 1 = dog, 0 =  cat
labelsc = [0 for addr in addrsc]

In [6]:
datad = []
for imagePath in addrsd:
# load the image, pre-process it, and store it in the data list
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    datad.append(img)

In [7]:
datac = []
for imagePath in addrsc:
# load the image, pre-process it, and store it in the data list
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    datac.append(img)

In [8]:
# to shuffle data
shuffle_data = True
if shuffle_data:
    d = list(zip(datad, labelsd))
    c = list(zip(datac, labelsc))
    e = d + c
    shuffle(e)
    data, labels = zip(*e)

In [9]:
del datad
del datac
del addrsd
del addrsc

In [10]:
Y_train = np.array(labels)
X_train = np.array(data, dtype="int8")

In [11]:
#preprocess for Resnet- 50
X_train =  preprocess_input(X_train)

In [12]:
# Two inputs one each - left and right image
left_input = Input((224,224,3))
right_input = Input((224,224,3))

In [13]:
#Import Resnetarchitecture from keras application and initializing each layer with pretrained imagenet weights.

'''
Please note that it’s usually better to intialize the layers with imagenet initializations than random. 
While training I will be updating the weights for each layer in each epoch. 
We don’t want to confuse this activity with transfer learning as 
I am not freezing any layer but initilializing each layer with imagenet weights
'''
convnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

c:\python\python37\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [14]:
# Add the final fully connected layers
x = convnet.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
preds = Dense(18, activation='sigmoid')(x) # Apply sigmoid
convnet = Model(inputs=convnet.input, outputs=preds)

In [15]:
#Applying above model for both the left and right images
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

In [16]:
# Euclidian Distance between the two images or encodings through the Resnet-50 architecture
Euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# use and add the distance function
Euc_distance = Euc_layer([encoded_l, encoded_r])

In [17]:
#identify the prediction
prediction = Dense(1,activation='sigmoid')(Euc_distance)

In [18]:
#Define the network with the left and right inputs and the ouput prediction
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

In [19]:
#define the optimizer. Here I have used SGD with nesterov momentum
optim = optimizers.SGD(lr=0.001, decay=.01, momentum=0.9, nesterov=True)

In [20]:
#compile the network using binary cross entropy loss and the above optimizer
siamese_net.compile(loss="binary_crossentropy",optimizer=optim,metrics=['accuracy'])

In [21]:
image_list = X_train[:180]
label_list = Y_train[:180]

In [22]:
left_input = []
right_input = []
targets = []

In [23]:
#Number of pairs per image
pairs = 8

In [24]:
#create the dataset to train on
for i in range(len(label_list)):
    for j in range(pairs):
        
        # we need to make sure that we are not comparing with the same image
        compare_to = i
        
        while compare_to == i: 
            compare_to = random.randint(0,179)
        
        left_input.append(image_list[i])
        right_input.append(image_list[compare_to])
        
        if label_list[i] == label_list[compare_to]:
            # if the images are same then label - 1
            targets.append(1.)
        else:
            # if the images are different then label - 0
            targets.append(0.)

In [25]:
#remove single-dimensional entries from the shape of the arrays and making them ready to create the train & datasets 
 
#the train data - left right images arrays and target label
left_input = np.squeeze(np.array(left_input))
right_input = np.squeeze(np.array(right_input))
targets = np.squeeze(np.array(targets))

In [26]:
# Creating test datasets - left, right images and target label
dog_image = X_train[4] #dog_image = 1, cat_image = 0

In [27]:
test_left = []
test_right = []
test_targets = []

In [28]:
for i in range(len(Y_train)-180):
    test_left.append(dog_image)
    test_right.append(X_train[i+180])
    test_targets.append(Y_train[i+180])

In [29]:
test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

In [32]:
#from keras_input_pipeline import *
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

session = tf.Session(config=config)

Instructions for updating:
non-resource variables are not supported in the long term


In [34]:
siamese_net.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 18)           126367634   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 18)           0           model_1[1][0]              

In [35]:
with tf.device('/gpu:1'):
    siamese_net.fit([left_input,right_input], targets,
          batch_size=16,
          epochs=30,
          verbose=1,
          validation_data=([test_left,test_right],test_targets))

RuntimeError: Attempting to capture an EagerTensor without building a function.